<a href="https://colab.research.google.com/github/Anik-et/Movie_Recommendation_System/blob/main/MovieRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Movie Recommendation System

### Importing the basic libraries

In [4]:
import numpy as np
import pandas as pd

### Importing & Parsing the dataset as ratings and movies details

In [8]:
ratingData = pd.io.parsers.read_csv('ratings.dat',
    names=['user_id', 'movie_id', 'rating', 'time'],
    engine='python', delimiter='::',encoding='ISO-8859-1')
movieData = pd.io.parsers.read_csv('movies.dat',
    names=['movie_id', 'title', 'genre'],
    engine='python', delimiter='::',encoding='ISO-8859-1')
print(ratingData)

         user_id  movie_id  rating       time
0              1      1193       5  978300760
1              1       661       3  978302109
2              1       914       3  978301968
3              1      3408       4  978300275
4              1      2355       5  978824291
...          ...       ...     ...        ...
1000204     6040      1091       1  956716541
1000205     6040      1094       5  956704887
1000206     6040       562       5  956704746
1000207     6040      1096       4  956715648
1000208     6040      1097       4  956715569

[1000209 rows x 4 columns]


### Create the ratings matrix of shape (m×u)

In [9]:
ratingMatrix = np.ndarray(
    shape=(np.max(ratingData.movie_id.values), np.max(ratingData.user_id.values)),
    dtype=np.uint8)
ratingMatrix[ratingData.movie_id.values-1, ratingData.user_id.values-1] = ratingData.rating.values
print(ratingMatrix)

[[  5 219 141 ...  39 205   3]
 [  0   0   0 ...  39 205  65]
 [  0   0   0 ...  41 205  65]
 ...
 [192  82 149 ... 126   0   0]
 [  0 136 152 ... 126   0   0]
 [ 64 253 155 ... 126   0   0]]


### Subtract Mean off - Normalization

In [10]:
normalizedMatrix = ratingMatrix - np.asarray([(np.mean(ratingMatrix, 1))]).T
print(normalizedMatrix)

[[ -48.76506623  165.23493377   87.23493377 ...  -14.76506623
   151.23493377  -50.76506623]
 [ -63.70612583  -63.70612583  -63.70612583 ...  -24.70612583
   141.29387417    1.29387417]
 [ -74.61854305  -74.61854305  -74.61854305 ...  -33.61854305
   130.38145695   -9.61854305]
 ...
 [  91.69039735  -18.30960265   48.69039735 ...   25.69039735
  -100.30960265 -100.30960265]
 [-100.35827815   35.64172185   51.64172185 ...   25.64172185
  -100.35827815 -100.35827815]
 [ -28.90794702  160.09205298   62.09205298 ...   33.09205298
   -92.90794702  -92.90794702]]


### Computing SVD

In [12]:
A = normalizedMatrix.T / np.sqrt(ratingMatrix.shape[0] - 1)
U, S, V = np.linalg.svd(A)

### Calculate cosine similarity, sort by most similar and return the top N

In [13]:
def similar(ratingData, movie_id, top_n):
    index = movie_id - 1 # Movie id starts from 1
    movie_row = ratingData[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', ratingData, ratingData)) #Einstein summation |  traditional matrix multiplication and is equivalent to np.matmul(a,b)
    similarity = np.dot(movie_row, ratingData.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity) #Perform an indirect sort along the given axis (Last axis)
    return sort_indexes[:top_n]

### Select k principal components to represent the movies, a movie_id to find recommendations and print the top_n results

In [14]:
k = 50
movie_id = 2
top_n = 5

sliced = V.T[:, :k] # representative data
indexes = similar(sliced, movie_id, top_n)

print('Recommendations for Movie {0}: \n'.format(
movieData[movieData.movie_id == movie_id].title.values[0]))
for id in indexes + 1:
    print(movieData[movieData.movie_id == id].title.values[0])

Recommendations for Movie Jumanji (1995): 

Jumanji (1995)
Beauty and the Beast (1991)
Indian in the Cupboard, The (1995)
Maverick (1994)
Jungle Book, The (1994)
